## **Import needed libraries**


In [1]:
import numpy as np
import random
import os
import time
from matplotlib import pyplot as plt
import pylab as pl

## **Cost function**
The cost function of the portfolio optimization problem 



#### Install packages 

In [2]:
pip install datapackage

distutils: /Users/einack/opt/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/einack/opt/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /Users/einack/opt/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/einack/opt/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/Users/einack/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pandas_datareader import data as web
from datapackage import Package
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.special import softmax

seed = 1000
np.random.seed(seed)

#### Investment parameters

In [4]:
num_stocks = 3 # Number of stocks to consider
total_price = 10000 # Total investment price
price_cap = 0.1*total_price # Maximum investment in each asset
min_trading_days = 20 # minimum trading days

#### Deriving stocks data from the SP 500 

In [5]:
package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')

# print processed tabular data (if exists any)
for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        data = np.array(resource.read())
data

array([['MMM', '3M', 'Industrials'],
       ['AOS', 'A. O. Smith', 'Industrials'],
       ['ABT', 'Abbott Laboratories', 'Health Care'],
       ...,
       ['ZBH', 'Zimmer Biomet', 'Health Care'],
       ['ZION', 'Zions Bancorp', 'Financials'],
       ['ZTS', 'Zoetis', 'Health Care']], dtype='<U38')

In [6]:
sp500_stocks = data[:, 0:1] # Select all stocks from the SP500
stocks = np.random.choice(sp500_stocks[:,0], num_stocks) # Choose randomly num_stocks out of 505

In [7]:
#Create a dataframe to store the adjusted close price of the stocks
df = pd.DataFrame()
#Store the adjusted close price of stock into the data frame

#Get the stock starting date
stockStartDate = '2020-12-01'
# Get the stocks ending date aka todays date and format it in the form YYYY-MM-DD
today = datetime.today().strftime('%Y-%m-%d')

for stock in stocks.tolist():
    df[stock] = web.DataReader(stock,data_source='yahoo',start=stockStartDate , end=today)['Adj Close']

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/TSLA/history?period1=1606813200&period2=1627372799&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n

In [ ]:
tmin = min_trading_days

In [ ]:
#Show the daily simple returns, NOTE: Formula = new_price/old_price - 1
returns = df.pct_change()
returns
#returns.to_numpy()[1:tmax, :]

In [ ]:
# Log-returns
log_returns = np.log(1.0 + returns)  #The assets forecast log-returns
logreturns = log_returns.to_numpy()[1:, :]

tmax = tmin + 1 #np.shape(logreturns)[0] # End time of the trading period


In [ ]:
# Assets covariance
log_returns.cov()

In [ ]:
T = tmax-tmin+1 # Total number of trading days
cov = np.zeros((T, num_stocks, num_stocks)) 
T

In [ ]:
# Computing the covariance matrix for each trading day
n = 0
for t in range(tmin, tmax+1):
  cv  = np.cov( logreturns[t, :].T ) 
  cov[n,:,:] = cv   
  n += 1
  
mu = logreturns[-T:] # The assets forecast returns starting from tmin

#### Compute function for metrics

In [ ]:
num_bits = np.rint(np.log2(total_price + 1.0 )).astype(int)
one      = np.ones((num_stocks))
encoding = [2**q for q in np.arange(num_bits)]

In [ ]:
# Hamiltonian
def H(config, cost_percentage=0.00001, risk_aversion=1.0, rho=1.0, tol=1e-10):

  samples = np.shape(config)[0]
  returns = np.zeros((samples))
  risk = np.zeros((samples))
  x = np.zeros((samples))
  cost_function = np.zeros((samples))
  deltaW = np.zeros((num_stocks, samples, T-1))

  w = np.sum(config.reshape(T * num_stocks * samples, num_bits) * encoding, axis=1)/total_price # Retrieving weight encoding from binary encoding
  w = w.reshape(num_stocks, samples, T)
  w1 = w.reshape(samples, num_stocks, T)

  for t in range(T-1):
    returns += np.dot(mu[t, :], w1[:, :, t].T) # It think it is ok
    risk += np.sum(np.multiply( w[:, :, t], np.dot(cov[t,:,:], w[:, :, t])), axis=0) # I think it is ok
    deltaW[:, :, t] = np.abs(w[:, :, t+1] - w[:, :, t])
    x += (np.dot(one, w[:,:,t]) -1.0 )**2

  # Includes the last trading day
  returns += np.dot(mu[T-1, :], w1[:, :, T-1].T) # It think it is ok
  risk += np.sum(np.multiply( w[:, :, T-1], np.dot(cov[T-1,:,:], w[:, :, T-1])), axis=0) # I think it is ok
  x += (np.dot(one, w[:,:,T-1]) -1.0 )**2

  x[x < tol] = 0.0
  
  cost_function = -returns + 0.5*risk_aversion*risk + cost_percentage*np.sum(deltaW, axis=(0,2)) + rho*x
  
  return cost_function

In [ ]:
def weights(x, num_stocks, num_bits, total_price=10000):
  
    samples = np.shape(x)[0]
    encoding = [2**q for q in np.arange(num_bits)]
    
    weight = np.sum(x.reshape(T * num_stocks * samples, num_bits) * encoding, axis=1)/total_price
    weight = weight.reshape(num_stocks, samples, T)
    
    return weight

In [ ]:
def Return(mu, w, time):
  """
            This function computes the overall log-returns
            ------------------------------------------------------------------------
            Parameters:
            mu:              np.array
                             a numpy array of dimension (T, num_stocks)
                             containing the daily log-returns
            w:               np.array
                             a numpy array of dimension (num_stocks, T)
                             containing the weights of each asset at a given time
            time:            int
                             Length of time of the trading period
            ------------------------------------------------------------------------
            Returns:         np.float
            returns          The overall return of the investment
            """
  samples = np.shape(w)[1]
  returns = np.zeros((samples))

  for t in range(time):
    returns += np.dot(mu[t, :], w[:, :, t])
  
  return -returns

In [ ]:
# This function computes the trading trajectory risk
def Risk(w, cov, time):
  
  samples = np.shape(w)[1]
  risk = np.zeros((samples))
  
  for t in range(time):
    risk += np.sum(np.multiply( w[:, :, t], np.dot(cov[t,:,:], w[:, :, t])), axis=0)  #np.dot( w[:,t].T, np.dot(cov[t,:,:], w[:, t]))

  return 0.5*risk

In [ ]:
# Transaction cost
# Cost prices is fixed for all asset during trading trajectory

def Transaction_cost(w, time, cost_percentage=0.00001):

  samples = np.shape(w)[1]
  deltaW = np.zeros((num_stocks, samples, time-1))

  for t in range(time-1):
    deltaW[:, :, t] = np.abs(w[:, :, t+1] - w[:, :, t])

  return cost_percentage*np.sum(deltaW, axis=(0,2))  #cost_percentage * np.sum(deltaW)

In [ ]:
# This function computes the Lagrange multiplier ensuring weights normalization
def normalization(w, time, tol=1e-10):
  samples = np.shape(w)[1]
  x = np.zeros((samples))

  for t in range(time):
    x += (np.dot(one, w[:,:,t]) -1.0 )**2

  x[x < tol] = 0.0

  return x